# Data Preparation

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

#Already implemented Fama-french factor model (Mkt-RF) into data below
data = pd.read_csv("Ken_French_Data.csv", header=0, index_col=0, na_values=-99.99)

rets = data/100
rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')

#Paper uses excess return, therefore need to take risk-free rate off industry returns
d1 = rets
d1 = d1.drop(['Mkt-RF','RF'], 1)
d1 = d1.sub(rets['RF'], axis =0)
d1["Mkt-RF"] = rets["Mkt-RF"]

rets = d1.loc["1963-07":"2004-11"]

# Function Preparation

In [2]:
#Paper uses mean and std for expected return and volatility calculation
def monthly_rets(r):
    return r.mean()

def monthly_vol(r):
    return r.std()

def portfolio_return(weights, returns):
    return weights.T @ returns

def portfolio_vol (weights, cov):
    return (weights.T @ cov @ weights)**0.5

#Risk free rate set to 0 as the data is already excess returns
def sharpe_ratio(r):
    sharpe = r.mean()/r.std()
    return sharpe

#Utility function for optimising
def util_fn(mean, cov):
    num_assets = cov.shape[0]
    inv_cov = np.linalg.pinv(cov)
    wtgs = (inv_cov@mean)/(np.ones((1,num_assets))@inv_cov@mean)
    return wtgs

# Preparing necessary functions for DM model (according to Sec A.2)

In [3]:
#In table 3 we can see that the sigma alpha is 0.01 therefore we can use this to find Tao
sigma_alpha = 0.01

def Shat2_fn(mu_hat_b, sigma_hat_bb):
    inv_cov = np.linalg.pinv(sigma_hat_bb)
    a = mu_hat_b.T @ inv_cov @ mu_hat_b
    return (a)
    
def tao_fn(sigma_alpha, sigma_hat):
    """
    Since,
    sigma_alpha = Tao * max(diag(var(epsilon)))
    
    Means that,
    annual(0.01) / max(diag(var(sigma_hat))) = Tao
    Therefore,
    Tao = (0.01/12)**2 / max(diag(sigma_hat))
    """
    a = (sigma_alpha/12)**2/np.diag(sigma_hat).max()
    return (a)

def w_fn(M, Shat2, tao):
    a = 1 + (int(M) * tao/(1+Shat2))
    return (1/a)

def delta_bar_fn(M, k, Shat2):
    a = (int(M) * (int(M) - 2) + k)/(int(M) * (int(M) - int(k) - 2)) - (int(k) + 3)/(int(M) * (int(M) - int(k) - 2)) * (Shat2/(1 + Shat2))
    return (a)

def delta_hat_fn(M, k):
    a = (int(M) - 2)*(int(M) + 1)/(int(M) * (int(M) - int(k) - 2))
    return(a)

def kappa_fn(M, k):
    a = (int(M) + 1)/(int(M) - int(k) - 2)
    return (a)

def h_fn(M, N):
    a = int(M)/(int(M) - int(N) - 1) 
    return(a)

def sigma_dm_aa_fn(kappa, w, beta_bar, beta_hat, sigma_hat_bb, delta_bar, delta_hat):
    a = kappa * ((w * beta_bar) + ((1 - w) * beta_hat)) @ sigma_hat_bb @ ((w * beta_bar) + ((1 - w) * beta_hat)).T + (h * (w * delta_bar + (1 - w) * delta_hat) * (w * sigma_bar + (1 - w) * sigma_hat))
    return (a)
    
def sigma_dm_ab_fn(w, beta_bar, beta_hat, sigma_hat_bb):
    a = kappa * ((w * beta_bar) + ((1 - w) * beta_hat)) @ sigma_hat_bb
    return (a)

def sigma_dm_bb_fn(kappa, sigma_hat_bb):
    a = kappa * sigma_hat_bb
    return (a)

# Loading CAPM data

In [4]:
#Risky assets
risky_assets = rets.drop(["Mkt-RF"], 1)
#Factor of CAPM being used in the paper which is the market risk premium
mrp = rets.loc["1963-07":"2004-11",['Mkt-RF']]

#Getting M and k
M = 120
n = pd.DataFrame(risky_assets).shape[1]


# Developing backtest system

In [5]:
#Developing weights

wtgs = []
for i in range(len(rets) - M + 1):
    #"Rolling window" data of 120 months
    df = (rets[i: i + M])
    #CAPM factor (Mkt-RF)
    mrp = pd.DataFrame(df["Mkt-RF"])
    k = pd.DataFrame(mrp).shape[1]
    #Only risky assets
    risky_assets = df.drop(["Mkt-RF"], 1)
    #Mu_hat_a
    mu_hat_a = risky_assets.mean()
    #Mean of the industries (Mu_hat_b)
    mu_hat_b = mrp.mean().values
    #Sigma_hat
    sigma_hat_bb = mrp.cov().values
    
    #Generating main parameters
    Shat2 = Shat2_fn(mu_hat_b, sigma_hat_bb)
    h = h_fn(M, n)
    kappa = kappa_fn(M, k)
    delta_hat = delta_hat_fn(M, k)
    delta_bar = delta_bar_fn(M, k, Shat2)
    
    #Running regressions
    #Intercept = 0 (restricted regression)
    mrp["alpha"] = 0
    beta_bar = []
    for x in risky_assets:
        lm = sm.OLS(risky_assets[x], mrp).fit()
        c = lm.params["Mkt-RF"]
        beta_bar.append(c)
    beta_bar = pd.DataFrame(beta_bar)
    #Intercept = 1 (unrestricted regression)
    mrp["alpha"] = 1
    mkt_df = []
    alpha_df = []
    for x in risky_assets:
        lm = sm.OLS(risky_assets[x], mrp).fit()
        c = lm.params["Mkt-RF"]
        d = lm.params["alpha"]
        mkt_df.append(c)
        alpha_df.append(d)
    factor_df = pd.DataFrame(mkt_df, columns = {"Mkt-RF"})
    alpha_df = pd.DataFrame(alpha_df)
    factor_df["alpha"] = alpha_df
    beta_hat = factor_df
    #beta_hat.columns = list(risky_assets)
    
    #Generating Sigma_hat and Sigma_bar
    #Factor returns
    #Estimation of sigma_hat
    df1 = pd.DataFrame(np.ones(mrp.shape[0]), index = df.index, columns =['alpha'])
    df1["Mkt-RF"] = mrp["Mkt-RF"] 
    factor_ret = df1.values @ beta_hat.T
    factor_ret.index = risky_assets.index
    factor_ret.columns = list(risky_assets)
    #Industries returns - Theoretical returns
    T_ret1 = risky_assets - factor_ret
    #Generating sigma_hat
    sigma_hat = np.cov(T_ret1.T)

    #Estimation of sigma_bar
    #Factor returns
    b = pd.DataFrame(mrp["Mkt-RF"]).values @ beta_bar.T
    b.index = risky_assets.index
    b.columns = list(risky_assets)
    #Industries returns - Theoretical returns
    T_ret2 = risky_assets - b
    #Generating sigma_bar
    sigma_bar = np.cov(T_ret2.T)
    
    #Generate tao, w and reassign beta_hat to only 'Beta' (remove alpha)
    tao = tao_fn(sigma_alpha, sigma_hat)
    w_value = w_fn(M, Shat2, tao)
    w = w_value
    beta_hat = pd.DataFrame(mkt_df)
    
    sigma_dm_aa = kappa * ((w * beta_bar) + ((1 - w) * beta_hat)) @ sigma_hat_bb @ ((w * beta_bar) + ((1 - w) * beta_hat)).T + (h * (w * delta_bar + (1 - w) * delta_hat) * (w * sigma_bar + (1 - w) * sigma_hat))
    sigma_dm_ab = kappa * ((w * beta_bar) + ((1 - w) * beta_hat)) @ sigma_hat_bb
    sigma_dm_bb = kappa * sigma_hat_bb
    
    mu_dm = w * np.concatenate([np.array(beta_bar) @ mu_hat_b, mu_hat_b]) + (1 - w) * np.concatenate([mu_hat_a, mu_hat_b])
    sigma_dm = np.concatenate([np.concatenate([sigma_dm_aa, sigma_dm_ab], axis=1), np.concatenate([sigma_dm_ab.T, sigma_dm_bb], axis=1)])
    
    weights_dm = util_fn(mu_dm, sigma_dm)
    wtgs.append(weights_dm)
    
dm_weights = pd.DataFrame(wtgs, index= rets[M-1:].index, columns = list(rets))

In [6]:
#Getting returns from the weightages
DM_rets = []
for Date, w in rets[M-1:].iterrows():
    wtgs = dm_weights.loc[Date].values  
    DM_rets.append(wtgs@rets.loc[Date].values) 
#Returns of portfolio
DM_rets =pd.DataFrame(DM_rets, index = rets[M-1:].index, columns =['Portfolio_return'])

In [7]:
sharpe_ratio(DM_rets)

Portfolio_return    0.038749
dtype: float64